In [ ]:
pip install geopandas

In [ ]:
pip install Shapely 

In [ ]:
pip install OSMPythonTools

In [ ]:
pip install openrouteservice

In [ ]:
pip install pyproj

In [ ]:
import os
import folium
import json 
import geopandas as gpd

In [ ]:
import shapely
from shapely.geometry import Point
from shapely.geometry import Polygon

from OSMPythonTools.overpass import overpassQueryBuilder, Overpass
from OSMPythonTools.nominatim import Nominatim

# Анализ пространственных данных. Домашнее задание №2

Мягкий дедлайн: __4 ноября 2020 г. 23:59__

Жесткий дедлайн (со штрафом в _50%_ от количества набранных вами за ДЗ баллов): __5 ноября 2020 г. 08:59__

Визуализация "чего-либо" __без__ выполненного основного задания оценивается в __0 баллов__

ФИО: `Мартиросян Завен Карапетович`

Группа: `DS 11`

## Задание №1. Горячая точка (алгоритм - 10 баллов, визуализация - 10 баллов).

Генерируйте рандомные точки на планете Земля до тех пор, пока не попадете на территорию ``Афганистана``

1. Вы можете использовать функции принадлжености точки полигону и расстояния от точки до полигона (в метрах)
2. Предложите не наивный алгоритм поиска (генерировать __напрямую__ точку из полигона границ Афганистана __запрещено__)

In [ ]:
!wget https://www.dropbox.com/s/w0t3944ljho8j1y/poly_afg.txt ## скачиваем подготовленный файл с полигоном Афганистана

In [ ]:
# Чтение файла и подготовка GDF  для дальнейшей визуализации
f = open("/content/poly_afg.txt")

afg_pol = []
while f.readline():
  coords = f.readline().split()
  afg_pol.append(Point(float(coords[0]), float(coords[1])))
afg_pol = Polygon(afg_pol)

gdf = gpd.GeoSeries(afg_pol, crs = "EPSG:4326")

In [ ]:
#Визаулизация полигона
m = folium.Map(
    location = afg_pol.centroid.coords[:][0][::-1],
    zoom_start = 2
)
folium.GeoJson(gdf.geometry).add_to(m)
folium.LayerControl().add_to(m)
m

In [ ]:
import random
import time

random.seed(21)           # воспроизводимость

m = folium.Map(
    location =  [45, -10],
    zoom_start = 3
)
folium.GeoJson(gdf.geometry).add_to(m)
folium.LayerControl().add_to(m)

x_min, y_min = -180, -180 # изначальная граница - весь мир
x_max, y_max = 180, 180   # изначальная граница - весь мир

new_x = random.uniform(x_min, x_max)
new_y = random.uniform(y_min, y_max)

new_point = Point(new_x, new_y)
my_icon = folium.map.Icon(color = 'black')
folium.Marker(location=[new_y, new_x],icon = my_icon, tooltip = str(0)).add_to(m)
my_colors  = ['black','black','darkred','darkred','darkred', 'darkblue','darkblue', 'darkblue', 'darkgreen','darkgreen','darkgreen',
              'blue','blue','blue','blue','lightred','lightred','lightred','beige','beige','beige','lightgreen']
i = 0
while afg_pol.contains(new_point) == False:

  if x_min / 2  < afg_pol.centroid.coords[0][0]:
    x_min = x_min / 2

  if x_max / 2  > afg_pol.centroid.coords[0][0]:
    x_max = x_max / 2

  if y_min / 2  < afg_pol.centroid.coords[0][1]:
    y_min = y_min / 2
  if y_max / 2  > afg_pol.centroid.coords[0][1]:
    y_max = y_max / 2
  
  new_x = random.uniform(x_min, x_max)
  new_y = random.uniform(y_min, y_max)
  new_point = Point(new_x, new_y)
  my_icon = folium.map.Icon(color = my_colors[i])
  folium.Marker(location=[new_y, new_x], icon = my_icon, tooltip = str(i+1), popup = str(i+1)).add_to(m)
  i = i + 1 

In [ ]:
m # При наведениии на маркер на вслывающем окошке написан номер по порядку появления данного маркера. По цветам - более темные маркеры появились раньше более светлых.

Визуализируйте пошагово предложенный алгоритм при помощи ``Folium``

## Задание №2. Качество жизни (20 баллов).

Для измерения показателя качества жизни в точке, найденной в предыдущем задании, вам необходимо рассчитать следующую сумму расстояний (в метрах):

1. Расстояние от точки до 5 ближайших __*__ банкоматов, находящихся в стране с наибольшим количеством объектов жилой недвижимости
2. Расстояние от точки до 5 ближайших школ, находящихся в стране с наибольшим количеством аптек в столице
3. Расстояние от точки до 5 ближайших кинотеатров, наодящихся в стране с самым большим отношением числа железнодорожных станций к автобусным остановкам в южной части __**__

__*__ При поиске _N_ ближайших объектов обязательно использовать ``R-tree``

__**__ Южной частью страны является территория, находящаяся к югу от множества точек, равноудаленных от самой северной и самой южной точек страны

## Задание №3. Поездка по Нью-Йорку (маршрут - 20 баллов, визуализация - 10 баллов).

Добраться __на автомобиле__ от входа в ``Central Park`` __Нью-Йорка__ (со стороны ``5th Avenue``) до пересечения ``Water Street`` и ``Washington Street`` в Бруклине (откуда получаются лучшие фото Манхэттенского моста) довольно непросто - разумеется, из-за вечных пробок. Однако еще сложнее это сделать, проезжая мимо школ, где дети то и дело переходят дорогу в неположенном месте.

Вам необходимо построить описанный выше маршрут, избегая на своем пути школы. Визуализируйте данный маршрут (также добавив школы и недоступные для проезда участки дорог) при помощи ``Folium``

Данные о расположении школ Нью-Йорка можно найти [здесь](https://catalog.data.gov/dataset/2019-2020-school-point-locations)

In [ ]:
import json
import requests

import folium
import pyproj
from pyproj import Transformer
from shapely import geometry
from shapely.geometry import Point, LineString, Polygon, MultiPolygon
from functools import partial
from pyproj import Proj, transform
from pyproj import CRS
from openrouteservice import client

In [ ]:
url = 'https://data.cityofnewyork.us/api/views/a3nt-yts4/rows.json'

def CreateBufferPolygon(point_in, resolution=10, radius=10):

    sr_wgs = pyproj.Proj(init='epsg:4326')
    sr_utm = pyproj.Proj(init='epsg:32632')
    point_in_proj = pyproj.transform(sr_wgs, sr_utm, *point_in)
    point_buffer_proj = Point(point_in_proj).buffer(radius, resolution=resolution)

    # Перебраем все точки в буфере и строим полигон
    poly_wgs = []
    for point in point_buffer_proj.exterior.coords:
        poly_wgs.append(pyproj.transform(sr_utm, sr_wgs, *point)) # Преобразуем обратно в WGS84

    return poly_wgs

In [ ]:
# (опционально) - игнорируем замечания при выполнении кода в Colab
import warnings
warnings.filterwarnings('ignore')

In [ ]:
api_key = '5b3ce3597851110001cf6248310884f9dd72498d96b00421fe1d2e0b' # Api key
clnt = client.Client(key = api_key) # создаем клиента
new_york_schools_json = requests.get(url).json() 

map_params = {'tiles': 'Stamen Terrain',
              'location': ([40.734425, -73.98094]),
              'zoom_start': 11}
map1 = folium.Map(**map_params)

site_coords = []
sites_poly = []

# Заполнить список полигонов школ
for school in new_york_schools_json["data"]:
    coord_y_x = school[8].split()
    site_coords.append([float(coord_y_x[1].strip("()")),
                        float(coord_y_x[2].strip("()"))])


for coords in site_coords:
    folium.features.Marker(list(reversed(coords)),
                           popup = 'School point<br>{0}'.format(coords)).add_to(map1)

    # Создание буферных полигонов вокруг школ
    site_poly_coords = CreateBufferPolygon(coords,
                                           resolution = 4,
                                           radius = 100)
    folium.vector_layers.Polygon(site_poly_coords).add_to(map1)
    sites_poly.append(site_poly_coords)

    site_poly_coords = [(y, x) for x, y in site_poly_coords]
    folium.vector_layers.Polygon(locations = site_poly_coords,
                                 color = '#ffd699',
                                 fill_color = '#ffd699',
                                 fill_opacity = 0.2,
                                 weight = 3).add_to(map1)
map1 # карта со всеми школами

In [ ]:
# Стиль для Geo_json'a
def style_function(color):
    return lambda feature: dict(color = color, weight = 3, opacity = 0.5)


map_params.update({ 'tiles': 'Stamen Terrain',
                    'location': ([40.734425, -73.98094]),
                    'zoom_start': 13})

# инициализация новой карты
map2 = folium.Map(**map_params)

coordinates_param = [[-73.97257, 40.76518],
                     [-73.9895824, 40.7031877]]
 
request_params = {'coordinates': coordinates_param,
                'format_out': 'geojson',
                'profile': 'driving-car',
                'preference': 'shortest',
                'instructions': 'false',}

route_normal = clnt.directions(**request_params)
folium.features.GeoJson(data = route_normal,
                        name = 'Route',
                        style_function = style_function('#FF0000'),
                        overlay = True).add_to(map2)
map2 # базовый маршрут

In [ ]:
# Буферный маршрут
route_buffer = LineString(route_normal['features'][0]['geometry']['coordinates']).buffer(0.12)

sites_buffer_poly = []
# Строим карту со школами, которые попадают в буферный полигон
for site_poly in sites_poly:
    poly = Polygon(site_poly)
    if route_buffer.intersects(poly):
        folium.features.Marker(list(reversed(poly.centroid.coords[0]))).add_to(map2)
        folium.vector_layers.PolyLine(site_poly).add_to(map2)
        folium.vector_layers.Polygon(site_poly,
                                 color = '#dddd00',
                                 fill_color = '#dddd00',
                                 fill_opacity = 0.8,
                                 weight = 5).add_to(map2)
        sites_buffer_poly.append(poly)
map2 #школы которые попвдвют в нашу буф. зону

In [ ]:
request_params['options'] = {'avoid_polygons': geometry.mapping(MultiPolygon(sites_buffer_poly))} #избегаем школы (+буффер), которые изображены выше
route_detour = clnt.directions(**request_params)

folium.features.GeoJson(data=route_detour, # маршрут в объезд школ
                        name='Route without schools',
                        style_function=style_function('#00FF00'),
                        overlay=True).add_to(map2)

map2.add_child(folium.map.LayerControl())


map2 # оптимизированный маршут в объезд школ
# В зависимости от школ, которые попали в список на "объезд" маршрут может получиться разный. 

# Красный маршрут - базовый, зеленый - в объезд полигонов школ. 
# При приближении карты можно заметить, что школы находятся на необходимом расстояниии от зеленого маршрута